# `swift_too` module

## Swift_PlanQuery example - querying planned *Swift* observations

### API version = 1.1

#### Author: Jamie A. Kennea (Penn State)

The Swift_PlanQuery class allows for querying the database of observations have have been planned for observation by Swift, otherwise known as the "Pre-Planned Science Timeline" (PPST). This allows the user to find out not only what was planned, but what will be observed by Swift in the future.

Two important caveats:

1. For past observations, what should have been down is mostly an academic exercise, as due to TOO uploads and GRBs, Swift's pre-planned observations can be obliterated. If you want to know what Swift *did* observe, do a Swift_ObsQuery

2. Swift's planning is only done days into the future, so if you want to know what is going to happen weeks from now, you're out of luck.

This therefore is mostly useful if you want to know what Swift is observing *today* and *tomorrow*.

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
from swifttools.swift_too import Swift_PlanQuery
from time import sleep
import numpy as np
from datetime import datetime,timedelta

### Setting up our username and secret key

Set up the class, giving our username and shared secret. These can be set up [on the Swift TOO website](https://www.swift.psu.edu/toop/too.php). After you log in, you will find your shared secret, and can modify it as necessary under the [Update Account Info](https://www.swift.psu.edu/toop/account_info.php) link. 

In [2]:
username = 'myuser'
shared_secret = 'mysharedsecret'

### First Query

First query example: What is Swift planned to observe now? Quite easy to construct. Here it is:

In [3]:
plan = Swift_PlanQuery(username,shared_secret,begin=datetime.utcnow())
_ = [print(ppt) for ppt in plan]

2021-04-05 17:38:00 - 2021-04-05 17:49:00 Target: SC1825          (00013984036) Exp:   660s


So the above should have reported back a single target. When you just give `begin` with no `end`, it simply returns the observation which is taking place at the time given. So what is happening at this time tomorrow?

In [4]:
plan = Swift_PlanQuery(username,shared_secret,begin=datetime.utcnow()+timedelta(days=1))
_ = [print(ppt) for ppt in plan]

2021-04-06 17:36:00 - 2021-04-06 17:42:00 Target: SGWGS-3940      (03108731002) Exp:   360s


Did the above query return anything? If not, then there simply isn't a plan for tomorrow. So how can we determine how far into the future Swift has a plan for? The `ppstmax` parameter. Let's check it out:

In [5]:
print(plan.ppstmax)

2021-04-07 00:00:00


This gives the date of the last planned target. If the query for what Swift is observing this time tomorrow didn't return anything, this date is probably the end of the UT day today. If the time you're running this is before midday on a Monday-Thursday, this is as expected. Typically the plan for tomorrow is submitted for upload around 12pm Eastern time. The exception to this rule is for the weekend plan. Typically on Thursday at 12pm we submit a plan for Friday and Saturday, and on Friday at 12pm we submit a plan for Sunday-Monday. This allows us to cover the weekend, while working a 5 day week.

### Second query example

Seeing a whole day's PPST. So let's see the whole day's Plan that goes farthest into the future.

In [6]:
dayplan = Swift_PlanQuery(username,shared_secret,begin=plan.ppstmax-timedelta(days=1),end=plan.ppstmax)
_ = [print(ppt) for ppt in dayplan]

2021-04-05 23:55:00 - 2021-04-06 00:00:00 Target: OGP_0860        (03100073005) Exp:   300s
2021-04-06 00:00:00 - 2021-04-06 00:05:00 Target: OGP_1073        (03101432003) Exp:   300s
2021-04-06 00:05:00 - 2021-04-06 00:09:21 Target: SMC J0103.0-7348 (00048714183) Exp:   261s
2021-04-06 00:09:21 - 2021-04-06 00:10:52 Target: SMC J0103.4-7109 (00048817169) Exp:    91s
2021-04-06 00:10:52 - 2021-04-06 00:12:23 Target: SMC J0034.8-7235 (00048764188) Exp:    91s
2021-04-06 00:12:23 - 2021-04-06 00:13:51 Target: SMC J0035.9-7329 (00048721190) Exp:    88s
2021-04-06 00:13:51 - 2021-04-06 00:15:19 Target: SMC J0036.5-7254 (00048750189) Exp:    88s
2021-04-06 00:15:19 - 2021-04-06 00:16:52 Target: SMC J0135.2-7347 (00048828124) Exp:    93s
2021-04-06 00:16:52 - 2021-04-06 00:18:23 Target: SMC J0054.5-7349 (00048712189) Exp:    91s
2021-04-06 00:18:23 - 2021-04-06 00:19:51 Target: SMC J0054.6-7424 (00048692189) Exp:    88s
2021-04-06 00:19:51 - 2021-04-06 00:21:21 Target: SMC J0126.5-7357 (0004

That's a lot of targets! Note that like Swift_ObsQuery it's showing you all the individual snapshots during the day. If you want to group these into observations, you can do that. Remember the `observations` method returns a dictionary, with obsid as the key, and the total observation details as the value.

In [7]:
_ = [print(observation) for observation in dayplan.observations.values()]

2021-04-05 23:55:00 - 2021-04-06 00:00:00 Target: OGP_0860        (03100073005) Exp:   300s Slewtime:     0s
2021-04-06 00:00:00 - 2021-04-06 00:05:00 Target: OGP_1073        (03101432003) Exp:   300s Slewtime:     0s
2021-04-06 00:05:00 - 2021-04-06 00:09:21 Target: SMC J0103.0-7348 (00048714183) Exp:   261s Slewtime:     0s
2021-04-06 00:09:21 - 2021-04-06 00:10:52 Target: SMC J0103.4-7109 (00048817169) Exp:    91s Slewtime:     0s
2021-04-06 00:10:52 - 2021-04-06 00:12:23 Target: SMC J0034.8-7235 (00048764188) Exp:    91s Slewtime:     0s
2021-04-06 00:12:23 - 2021-04-06 00:13:51 Target: SMC J0035.9-7329 (00048721190) Exp:    88s Slewtime:     0s
2021-04-06 00:13:51 - 2021-04-06 00:15:19 Target: SMC J0036.5-7254 (00048750189) Exp:    88s Slewtime:     0s
2021-04-06 00:15:19 - 2021-04-06 00:16:52 Target: SMC J0135.2-7347 (00048828124) Exp:    93s Slewtime:     0s
2021-04-06 00:16:52 - 2021-04-06 00:18:23 Target: SMC J0054.5-7349 (00048712189) Exp:    91s Slewtime:     0s
2021-04-06 0

You'll note that the slewtime is zero for each observation. This is because slewtime isn't reported for pre-planned targets.